In [2]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
import os

C:\Users\Anubha\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.constraints import maxnorm
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

C:\Users\Anubha\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
#using OpenCV library of python


In [4]:
#With just image flatened
def image_to_feature_vector(image, size=(128, 128)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

#With histogram graph plot of image , normalised 
def extract_color_histogram(image, bins=(32, 32, 32)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])

	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)

	# otherwise, perform "in place" normalization in OpenCV 3
	else:
		cv2.normalize(hist, hist)

	# return the flattened histogram as the feature vector
	return hist.flatten()


In [5]:
train_data_folder=['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna']
#Label = ['Ka','Kha','Ga','Gha','Kna']
Labels = [0,1,2,3,4]

In [9]:
cd ..

C:\Users\Anubha\Desktop


In [10]:

    
def load_photos(train_data_folder,Desktop):
    i=0
    y=[]
    images=[]
    rawImages = []
    features = []
    for directory in train_data_folder:
        y_val=Labels[i]
        
        for name in listdir(Desktop+'/'+ directory):
            # load an image from file
            filename = Desktop+'/'+directory + '/' + name
            image = cv2.imread(filename) #target_size=(32, 32,1))
            # convert the image pixels to a numpy array
            pixels = image_to_feature_vector(image)
            hist = extract_color_histogram(image)

            rawImages.append(pixels)
            features.append(hist)
            #labels.append(label)
            y.append(y_val)
        i=i+1;
    return rawImages,features,y

# load images
Train_Desktop = 'Train_val'
rawimages,features,y = load_photos(train_data_folder,Train_Desktop)
print('Loaded Images: %d' % len(rawimages))


Loaded Images: 8500


In [11]:
#Converting in numpy array
rawImages = np.array(rawimages)
features = np.array(features)

In [12]:
labels = np.array(y)

In [13]:
len(labels)

8500

In [14]:
print("[INFO] pixels matrix: {:.2f}MB".format(
	rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(
	features.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 408.00MB
[INFO] features matrix: 1088.00MB


In [15]:
Desktop_test = 'Test'
rawimages_y,features_y,y_y = load_photos(train_data_folder,Desktop_test)
print('Loaded Images Test: %d' % len(rawimages_y))

Loaded Images Test: 1500


In [34]:
len(features)

8500

In [16]:
#y_test features : raw and histogram version
rawImages_y = np.array(rawimages_y)
features_y = np.array(features_y)
labels_y= np.array(y_y)

In [35]:
# Shuffling data : Histogram Train Data
f = np.c_[features.reshape(len(features), -1), labels.reshape(len(labels), -1)]
np.random.shuffle(f)

features1 = f[:, :features.size//len(features)].reshape(features.shape)
labels_f1 = f[:, features.size//len(features):].reshape(labels.shape)

In [39]:
#Shuffling Test Histogram data
fy = np.c_[features_y.reshape(len(features_y), -1), labels_y.reshape(len(labels_y), -1)]
np.random.shuffle(fy)

features_y1 = fy[:, :features_y.size//len(features_y)].reshape(features_y.shape)
labels_fy1 = fy[:, features_y.size//len(features_y):].reshape(labels_y.shape)

In [24]:
#Shuffling Train Raw Image data
c = np.c_[rawImages.reshape(len(rawImages), -1), labels.reshape(len(labels), -1)]
np.random.shuffle(c)

rawImages1 = c[:, :rawImages.size//len(rawImages)].reshape(rawImages.shape)
labels1 = c[:, rawImages.size//len(rawImages):].reshape(labels.shape)

In [26]:
#Shuffling Test Raw Image data
cy = np.c_[rawImages_y.reshape(len(rawImages_y), -1), labels_y.reshape(len(labels_y), -1)]
np.random.shuffle(cy)

rawImages_y1 = cy[:, :rawImages_y.size//len(rawImages_y)].reshape(rawImages_y.shape)
labels_y1 = cy[:, rawImages_y.size//len(rawImages_y):].reshape(labels_y.shape)

In [37]:
print(len(rawImages),len(rawImages1),len(features1))

8500 8500 8500


In [41]:
#neural network
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=.1)
model.fit(rawImages1, labels1)
acc = model.score(rawImages_y1, labels_y1)
print("[INFO] neural network raw pixel accuracy: {:.2f}%".format(acc * 100))


#neural network
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, alpha=1e-4,
                      solver='sgd', tol=1e-4, random_state=1,
                      learning_rate_init=.1)
model.fit(features1, labels_f1)
acc = model.score(features_y1, labels_fy1)
print("[INFO] neural network histogram accuracy: {:.2f}%".format(acc * 100))



[INFO] evaluating raw pixel accuracy...
[INFO] neural network raw pixel accuracy: 20.00%


[INFO] evaluating histogram accuracy...
[INFO] neural network histogram accuracy: 36.47%


In [44]:
# k-NN
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = KNeighborsClassifier(n_neighbors=5)
model.fit(rawImages1, labels1)
acc = model.score(rawImages_y1, labels_y1)

#either faulty or exceptional 
print("[INFO] neural network raw pixel accuracy: {:.2f}%".format(acc * 100))

# k-NN
print("\n")
print("[INFO] evaluating histogram accuracy...")
model = KNeighborsClassifier(n_neighbors=5)
model.fit(features1, labels_f1)
acc = model.score(features_y1, labels_fy1)
print("[INFO] neural network histogram accuracy: {:.2f}%".format(acc * 100))



[INFO] evaluating raw pixel accuracy...
[INFO] neural network raw pixel accuracy: 98.87%


[INFO] evaluating histogram accuracy...
[INFO] neural network histogram accuracy: 38.33%


In [45]:

#SVC
print("\n")
print("[INFO] evaluating raw pixel accuracy...")
model = SVC(max_iter=1000,class_weight='balanced')
model.fit(rawImages1, labels1)
acc = model.score(rawImages_y1, labels_y1)
print("[INFO] SVM-SVC raw pixel accuracy: {:.2f}%".format(acc * 100))
#SVC
print("\n")
print("[INFO] evaluating histogram accuracy...")
#default kernel RBS
model = SVC(max_iter=1000,class_weight='balanced')
model.fit(features1, labels_f1)
acc = model.score(features_y1, labels_fy1)
print("[INFO] SVM-SVC histogram accuracy: {:.2f}%".format(acc * 100))




[INFO] evaluating raw pixel accuracy...


C:\Users\Anubha\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[INFO] SVM-SVC raw pixel accuracy: 20.00%


[INFO] evaluating histogram accuracy...
[INFO] SVM-SVC histogram accuracy: 21.87%
